In [2]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

In [7]:
# Read all the US state names with codes
def state_names_data() -> pd.core.frame.DataFrame:
    states_and_codes_filename = 'data/states.csv'
    states_df = pd.read_csv(states_and_codes_filename)[["State","Code"]]
    states_df = states_df.sort_values(by=['State'])
    return states_df[states_df["State"].isin(["Alaska", "New York"])]

# Test case for states_names_data()
assert state_names_data().shape == (2, 2)

In [8]:
# Read the resident populations data for all states
def state_populations_data() -> pd.core.frame.DataFrame:
    state_populations_filename = 'data/population-nst-est2020.xlsx'
    state_populations_sheetname = 'NST01'
    state_populations_df = pd.read_excel(state_populations_filename, sheet_name=state_populations_sheetname, header=None, skiprows=9, skipfooter=8)
    state_populations_df = state_populations_df[[0,14]].replace(to_replace=r'^\.', value='', regex=True).rename(columns={0:"State", 14:"Population"})
    state_populations_df = pd.merge(state_populations_df, state_names_data(), left_on="State", right_on="State", how="inner")
    state_populations_df = state_populations_df.sort_values(by=['State'])
    return state_populations_df[state_populations_df["State"].isin(["Alaska", "New York"])]

assert state_populations_data().shape == (2, 3)

In [9]:
# Read the COVID-19 cases, deaths, and testing data for all states
def c_infection_rate_data() -> pd.core.frame.DataFrame:
    infection_rate_filename = 'data/united_states_covid19_cases_deaths_and_testing_by_state.csv'
    infection_rate_df = pd.read_csv(infection_rate_filename, header=2)[
        ["State/Territory", "Cases in Last 7 Days"]]

    # Counts for New York City and New York State are shown separately for case and death metrics
    ny_state_cases = infection_rate_df[infection_rate_df["State/Territory"] == "New York*"]
    nyc_cases = infection_rate_df[infection_rate_df["State/Territory"] == "New York City"]
    ny_cases = float(ny_state_cases["Cases in Last 7 Days"]) + float(
        nyc_cases["Cases in Last 7 Days"])

    # Add row for NY
    infection_rate_df = infection_rate_df.append(pd.DataFrame([["New York", ny_cases]],
                                                              columns=["State/Territory",
                                                                       "Cases in Last 7 Days"]))

    infection_rate_df = pd.merge(infection_rate_df, state_names_data(), left_on="State/Territory", right_on="State",
             how='inner')[["State", "Cases in Last 7 Days", "Code"]].sort_values(by=['State'])
    
    infection_rate_df["Infection_Rate_Percentage"] = infection_rate_df["Cases in Last 7 Days"] / infection_rate_df["Cases in Last 7 Days"].sum()
    
    return infection_rate_df[infection_rate_df["State"].isin(["Alaska", "New York"])]

assert c_infection_rate_data().shape == (2, 4)

In [10]:
# Read the vaccinated and unvaccinated population for all states
def p_unvaccinated_population_data() -> pd.core.frame.DataFrame:
    unvax_population_filename = 'data/covid19_vaccinations_in_the_united_states.csv'
    unvax_population_df = pd.read_csv(unvax_population_filename, header=2)[["State/Territory/Federal Entity", "People with at least One Dose by State of Residence"]]
    unvax_population_df = unvax_population_df.replace(to_replace=r'^New York State$', value='New York', regex=True)
    unvax_population_df = pd.merge(unvax_population_df, state_populations_data(), left_on="State/Territory/Federal Entity", right_on="State", how="inner")
    unvax_population_df = unvax_population_df.rename(columns={"People with at least One Dose by State of Residence":"Vax_Population"})
    unvax_population_df["Unvax_Population"] = unvax_population_df["Population"] - unvax_population_df["Vax_Population"]
    unvax_population_df["Unvax_Population_Percentage"] = unvax_population_df["Unvax_Population"] / unvax_population_df["Unvax_Population"].sum()
    unvax_population_df = unvax_population_df[["State", "Population", "Vax_Population", "Unvax_Population", "Unvax_Population_Percentage"]].sort_values(by=['State'])
    return unvax_population_df[unvax_population_df["State"].isin(["Alaska", "New York"])]

assert p_unvaccinated_population_data().shape == (2, 5)

In [12]:
# Read the 7-day average vaccination rates / 100k people for each state
def u_vaccination_rate_data(index: int) -> pd.core.frame.DataFrame:
    vaccination_rate_filename = f'data/trends_in_number_of_covid19_vaccinations_in_the_us ({index}).csv'
    vaccination_rate_df = pd.read_csv(vaccination_rate_filename, header=2)
    vaccination_rate_df = vaccination_rate_df[vaccination_rate_df["Date Type"] == "Admin"]
    vaccination_rate_df = vaccination_rate_df[["Date", "Location", "7-Day Avg Total Doses Daily"]]
    return vaccination_rate_df


def u_vaccination_data() -> pd.core.frame.DataFrame:
    vaccination_df = u_vaccination_rate_data(1)

    for i in range(2, 52):
        vaccination_df = vaccination_df.append(u_vaccination_rate_data(i))

    vaccination_df = vaccination_df[vaccination_df["Date"] == vaccination_df["Date"].max()]

    vaccination_df = pd.merge(vaccination_df, state_populations_data(), left_on="Location",
                              right_on="Code", how="inner")

#     vaccination_df["7_Day_Daily_Vaccination_Rate"] = vaccination_df["7-Day Avg Total Doses Daily"] / vaccination_df["Population"]

#     vaccination_df["7_Day_Daily_Vaccination_Rate_Percentage"] = vaccination_df[
#                                                                     "7_Day_Daily_Vaccination_Rate"] * 100 / \
#                                                                 vaccination_df[
#                                                                     "7_Day_Daily_Vaccination_Rate"].sum()
    
#     vaccination_df["7_Day_Daily_Vaccination_Rate_Reciprocal"] = 1 / vaccination_df["7_Day_Daily_Vaccination_Rate"]

    vaccination_df = vaccination_df[["State", "Code", "7-Day Avg Total Doses Daily"]].sort_values(by=['State'])

    return vaccination_df[vaccination_df["State"].isin(["Alaska", "New York"])]


assert u_vaccination_data().shape == (2, 3)

In [13]:
# define the constraints

vT = 1000 # Total number of vaccines


In [14]:
def create_constraints_coeffs() -> np.ndarray:
    upper_bound_coefficients = np.identity(2)
    lower_bound_coefficients = np.diag(np.ones(2)*-1)
    total_bound_coefficients = np.array([np.ones(2)])

    upper_and_lower_bounds = np.concatenate((upper_bound_coefficients, lower_bound_coefficients), axis=0)

    return np.concatenate((total_bound_coefficients, upper_and_lower_bounds), axis=0)

assert create_constraints_coeffs().shape == (5, 2)

In [15]:
def create_bounds_df() -> pd.core.frame.DataFrame:
    p_unvaccinated_population_df = p_unvaccinated_population_data()
    p_unvaccinated_population_df["Vaccine_Distribution_By_Unvax_Population"] = (p_unvaccinated_population_df["Unvax_Population_Percentage"] * vT).astype('int64')
    p_unvaccinated_population_df = p_unvaccinated_population_df[["State", "Vaccine_Distribution_By_Unvax_Population"]]

    c_infection_data_df = c_infection_rate_data()
    c_infection_data_df["Vaccine_Distribution_By_Infection_Rate"] = (c_infection_data_df["Infection_Rate_Percentage"] * vT).astype('int64')
    c_infection_data_df = c_infection_data_df[["State", "Vaccine_Distribution_By_Infection_Rate"]]

    return pd.merge(p_unvaccinated_population_df, c_infection_data_df, left_on="State", right_on="State", how="inner")

def create_bounds() -> np.ndarray:
    bounds_df = create_bounds_df()
    
    bounds_df["Upper_Bound"] = bounds_df[["Vaccine_Distribution_By_Unvax_Population", "Vaccine_Distribution_By_Infection_Rate"]].max(axis=1)
    bounds_df["Lower_Bound"] = bounds_df[["Vaccine_Distribution_By_Unvax_Population", "Vaccine_Distribution_By_Infection_Rate"]].min(axis=1) * -1
    bounds = np.ones(1) * vT
    bounds = np.concatenate((bounds, bounds_df["Upper_Bound"].to_numpy().transpose()), axis=0)

    return np.concatenate((bounds, bounds_df["Lower_Bound"].to_numpy().transpose()), axis=0)

assert create_bounds().shape == (5,)

In [16]:
def create_obj_coeffs() -> np.ndarray:
    return u_vaccination_data()["7-Day Avg Total Doses Daily"].to_numpy().transpose()

assert create_obj_coeffs().shape == (2,)

In [17]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = create_constraints_coeffs()
    data['bounds'] = create_bounds()
    data['obj_coeffs'] = create_obj_coeffs()
    data['num_vars'] = 2
    data['num_constraints'] = 5
    return data

# def create_data_model():
#     """Stores the data for the problem."""
#     data = {}
#     data['constraint_coeffs'] = [
#         [1, 1], 
#         [1, 0],
#         [0, 1],
#         [-1, -0],
#         [-0, -1]
#     ]
#     data['bounds'] = [100000,  73048.0, 94908.0, -5091.0, -26951.0]
#     data['obj_coeffs'] = [ 801 * 100000 / 731338, 34831 * 100000 / 19299981 ]
#     data['num_vars'] = 2
#     data['num_constraints'] = 5
#     return data

In [18]:
data = create_data_model()

# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, infinity, 'x[%i]' % j)
print('Number of variables =', solver.NumVariables())

Number of variables = 2


In [19]:
# for i in range(data['num_constraints']):
#     constraint = solver.RowConstraint(0, data['bounds'][i], '')
#     for j in range(data['num_vars']):
#         constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
# print('Number of constraints =', solver.NumConstraints())
# In Python, you can also set the constraints as follows.
for i in range(data['num_constraints']):
 constraint_expr = \
[data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
 solver.Add(sum(constraint_expr) <= data['bounds'][i])

In [20]:
# objective = solver.Objective()
# for j in range(data['num_vars']):
#     objective.SetCoefficient(x[j], data['obj_coeffs'][j])
# objective.SetMaximization()
# In Python, you can also set the objective as follows.
obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
solver.Maximize(solver.Sum(obj_expr))

In [21]:
status = solver.Solve()

In [22]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    total = 0
    for j in range(data['num_vars']):
        print(x[j].name(), ' = ', x[j].solution_value())
        total += x[j].solution_value()
    print()
    print('Total =', total)
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 33061440.0
x[0]  =  52.0
x[1]  =  948.0

Total = 1000.0
Problem solved in 3683.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes


In [23]:
# print(unvaccinated_population_data())
# print(infection_rate_data())

results_df = pd.merge(p_unvaccinated_population_data()[["State", "Population", "Vax_Population", "Unvax_Population"]], c_infection_rate_data()[["State", "Cases in Last 7 Days"]], left_on="State", right_on="State", how="inner")


# print(vaccination_data())

results_df = pd.merge(results_df, u_vaccination_data()[["State", "7-Day Avg Total Doses Daily"]], on="State", how="inner")

results_df = pd.merge(results_df, create_bounds_df(), on="State", how="inner")

solutions_array = np.array(list(map(lambda y: y.solution_value(), list(x.values()))))

results_df["Optimal_Distribution"] = solutions_array

results_df.to_csv('results.csv', index=False)

In [24]:
print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 5
   Variables        : 2
     Binary         : 0
     Integer        : 2
     Continuous     : 0
Maximize
 Obj: +801 x[0] +34831 x[1] 
Subject to
 auto_c_000000000: +1 x[0] +1 x[1]  <= 1000
 auto_c_000000001: +1 x[0]  <= 105
 auto_c_000000002: +1 x[1]  <= 948
 auto_c_000000003: -1 x[0]  <= -51
 auto_c_000000004: -1 x[1]  <= -894
Bounds
 0 <= x[0] <= inf
 0 <= x[1] <= inf
Generals
 x[0]
 x[1]
End

